In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import math
import seaborn as sns

# 경고 메시지 숨기기
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("./data/kkbox_data_total.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955126 entries, 0 to 955125
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   msno                    955126 non-null  int64  
 1   city                    955126 non-null  int64  
 2   bd                      955126 non-null  int64  
 3   gender                  440263 non-null  object 
 4   registered_via          955126 non-null  int64  
 5   registration_init_time  955126 non-null  int64  
 6   is_churn                955126 non-null  int64  
 7   is_back                 955126 non-null  int64  
 8   payment_plan_sum        955126 non-null  int64  
 9   plan_list_price         955126 non-null  int64  
 10  actual_amount_paid      955126 non-null  int64  
 11  discount_rate           954132 non-null  float64
 12  is_auto_renew           955126 non-null  float64
 13  membership_expire_date  955126 non-null  int64  
 14  is_cancel           

### 우중님 전처리

In [3]:

aa = df[df['payment_plan_sum'].isin([0, 2882, 3550])].index
df.drop(aa, inplace=True)
bb = df[df['plan_list_price'].isin([0, 13186, 16800, 17433])].index
df.drop(bb, inplace=True)
cc = df[df['plan_list_price'].isin([0, 13186, 16800, 17433])].index
df.drop(cc, inplace=True)

In [4]:

aa = df[df['num_25'] >= 20000].index
df.drop(aa, inplace=True)
bb = df[df['num_50'] >= 30000].index
df.drop(bb, inplace=True)
cc = df[df['num_75'] >= 10000].index
df.drop(cc, inplace=True)
dd = df[df['num_985'] >= 20000].index
df.drop(dd, inplace=True)
ee = df[df['num_100'] >= 35000].index
df.drop(ee, inplace=True)
ff = df[df['num_unq'] >= 24000].index
df.drop(ff, inplace=True)
gg = df[df['total_secs'] < 0].index
df.drop(gg, inplace=True)
df.drop(df[df['msno'] == 4524786].index, inplace=True)

#### 다인 전처리

In [5]:
def int_to_date(df):
    # 정수형을 문자열로 변환한 후 날짜 형식으로 변환하고, Pandas datetime64 형식으로 변환
    df['membership_expire_date'] = pd.to_datetime(df['membership_expire_date'].astype(str), format='%Y%m%d', errors='coerce')
    df['registration_init_time'] = pd.to_datetime(df['registration_init_time'].astype(str), format='%Y%m%d', errors='coerce')
    return df

In [6]:
def discount_rate_preprocessing(df):
    df.drop(df[(df["actual_amount_paid"] != 0) & df["discount_rate"].isna()].index, inplace=True)
    df["discount_rate"].fillna(0, inplace=True)
    return df

In [ ]:
def membership_expire_date_preprocessing(df):
    # membership_expire_date가 registration_init_time보다 빠른 행을 필터링
    df_cleaned = df[df['membership_expire_date'] >= df['registration_init_time']]

    start_date = pd.to_datetime("2005-01-01")
    end_date = pd.to_datetime("2018-04-30")

    # membership_expire_date가 날짜 범위 밖에 있는 데이터를 필터링
    df_cleaned = df_cleaned[(df_cleaned['membership_expire_date'] >= start_date) & (df_cleaned['membership_expire_date'] <= end_date)]
    return df_cleaned

In [8]:
def remove_outliers_discount_rate(df):
    # discount_rate가 0보다 작거나 1보다 큰 이상치 제거
    df_cleaned = df[(df["discount_rate"] >= 0) & (df["discount_rate"] <= 1)]
    return df_cleaned

In [9]:
int_to_date(df)
discount_rate_preprocessing(df)

,msno,city,bd,gender,registered_via,registration_init_time,is_churn,is_back,payment_plan_sum,plan_list_price,...,transaction_count,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,log_start,log_end
1,6597367,4,30,male,9,2011-09-16,0,0,603,2980,...,21,77,22,16,44,658,287,171508.646,20150209,20170331
2,3629318,5,34,male,9,2011-09-16,1,0,360,1072,...,2,145,84,58,42,1678,1158,448313.054,20160822,20170330
3,2120253,13,63,male,9,2011-09-18,0,0,783,3874,...,27,1173,390,246,308,5003,5784,1435958.164,20150103,20170330
4,2836580,1,0,NaN,7,2011-09-18,1,0,780,3874,...,27,830,252,197,276,3326,3543,982573.945,20150101,20170328
5,661240,4,28,female,9,2011-09-20,0,0,270,1589,...,10,81,20,12,16,438,343,107171.103,20150301,20170331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955121,4441402,1,0,NaN,7,2015-10-20,0,0,540,1782,...,18,356,254,115,75,700,988,253640.163,20151020,20170331
955122,2788470,1,0,NaN,7,2015-10-20,0,0,540,2682,...,18,4662,680,544,746,13437,16732,3658622.489,20151020,20170331
955123,2755294,1,0,NaN,7,2015-10-20,0,0,540,1782,...,18,439,91,56,57,7421,4749,1376617.577,20151023,20170331
955124,6133792,6,26,female,7,2015-10-20,0,0,570,2681,...,19,10890,1055,631,513,6457,15314,1954030.130,20151020,20170331


In [10]:
df = membership_expire_date_preprocessing(df)

In [11]:
df = remove_outliers_discount_rate(df)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 856566 entries, 1 to 955125
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   msno                    856566 non-null  int64         
 1   city                    856566 non-null  int64         
 2   bd                      856566 non-null  int64         
 3   gender                  368210 non-null  object        
 4   registered_via          856566 non-null  int64         
 5   registration_init_time  856566 non-null  datetime64[ns]
 6   is_churn                856566 non-null  int64         
 7   is_back                 856566 non-null  int64         
 8   payment_plan_sum        856566 non-null  int64         
 9   plan_list_price         856566 non-null  int64         
 10  actual_amount_paid      856566 non-null  int64         
 11  discount_rate           856566 non-null  float64       
 12  is_auto_renew           856566 non-

In [16]:
df.describe()

,msno,city,bd,registered_via,registration_init_time,is_churn,is_back,payment_plan_sum,plan_list_price,actual_amount_paid,...,transaction_count,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,log_start,log_end
count,8.565660e+05,856566.000000,856566.000000,856566.000000,856566,856566.000000,856566.000000,856566.000000,856566.000000,856566.000000,...,856566.000000,856566.000000,856566.000000,856566.000000,856566.000000,856566.000000,856566.000000,8.565660e+05,8.565660e+05,8.565660e+05
mean,3.384144e+06,5.682107,12.870459,6.817386,2013-12-01 00:12:19.765061120,0.126265,0.005396,485.846328,2160.963544,2223.718082,...,15.892080,1448.593952,368.655433,229.146750,246.097319,5973.356396,6115.638805,1.405346e+11,2.015515e+07,2.016942e+07
min,1.200000e+01,1.000000,-3152.000000,3.000000,2004-03-26 00:00:00,0.000000,0.000000,7.000000,1.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000e-03,2.015010e+07,2.015010e+07
25%,1.693459e+06,1.000000,0.000000,7.000000,2012-05-23 00:00:00,0.000000,0.000000,270.000000,1089.000000,1089.000000,...,8.000000,249.000000,67.000000,42.000000,42.000000,1055.000000,1233.000000,2.981190e+05,2.015010e+07,2.017032e+07
50%,3.383370e+06,1.000000,0.000000,7.000000,2014-11-02 00:00:00,0.000000,0.000000,510.000000,1980.000000,1968.000000,...,16.000000,785.000000,211.000000,136.000000,138.000000,3795.000000,4142.000000,1.041258e+06,2.015100e+07,2.017033e+07
75%,5.073124e+06,12.000000,26.000000,9.000000,2016-02-14 00:00:00,0.000000,0.000000,690.000000,3222.000000,3427.000000,...,23.000000,1940.000000,501.000000,318.000000,331.000000,9021.000000,9474.000000,2.435046e+06,2.016061e+07,2.017033e+07
max,6.769459e+06,22.000000,2016.000000,13.000000,2017-03-31 00:00:00,1.000000,1.000000,2032.000000,8615.000000,8614.000000,...,82.000000,19820.000000,12235.000000,7910.000000,16645.000000,34998.000000,23999.000000,9.223372e+15,2.017033e+07,2.017033e+07
std,1.953189e+06,6.375604,19.333008,1.947062,NaN,0.332148,0.073259,249.061807,1234.423067,1292.169517,...,8.825122,1826.431092,460.231445,277.117868,328.737921,6294.587002,5954.767038,3.593533e+13,6.172958e+03,3.285733e+03


In [17]:
df.to_csv("data_total.csv")